In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from tqdm import tqdm

POSTGRES_ADDRESS = 'db.panoply.io' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5439'
POSTGRES_USERNAME = 'vin.tang@gmail.com' ## CHANGE THIS TO YOURPANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'Cobalt27!' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD
POSTGRES_DBNAME = 'playerslounge_prod' ## CHANGE THIS TO YOUR DATABASE NAME

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,
password=POSTGRES_PASSWORD,
ipaddress=POSTGRES_ADDRESS,
port=POSTGRES_PORT,
dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

In [2]:
query = """
select contests.id, contests.__updatetime, contests.type, contests_plrsbefore.plr, contests.winner, contests.loser, contests_plrsbefore.userid, contests.status
from contests 
inner join contests_plrsbefore 
on contests.id = contests_plrsbefore.contests_id
where contests.console_game = 'ps4-madden21'
"""

In [3]:
df = pd.read_sql_query(query, cnx)
df.shape

(333298, 8)

In [4]:
df.head()

,id,__updatetime,type,plr,winner,loser,userid,status
0,7995092012829590,2020-08-21 19:15:34.342,H,64,4.439926e+14,7.038696e+15,443992607815436,completed
1,5179645355963421,2020-08-21 20:10:45.923,H,59,7.893768e+15,6.613301e+15,6613300821617026,completed
2,5484846488566522,2020-08-21 20:10:45.923,H,64,NaN,NaN,443992607815436,cancelled
3,2230477145682123,2020-08-21 21:13:48.940,H,69,NaN,NaN,1768930121316690,cancelled
4,3214806079020904,2020-08-21 22:19:37.525,H,62,NaN,NaN,4855353181157037,cancelled


In [17]:
lookup = {}

for _, row in tqdm(df.iterrows()):
    if row['status'] != "completed" or row['type'] != 'H':
        pass
    else:
        if row['userid'] not in lookup.keys():
            if row['userid'] == row['winner']:
                lookup[row['userid']] = [(row['__updatetime'], 1)]
            else:
                lookup[row['userid']] = [(row['__updatetime'], 0)]
        else:
            if row['userid'] == row['winner']:
                lookup[row['userid']].append((row['__updatetime'], 1))
            else:
                lookup[row['userid']].append((row['__updatetime'], 0))

333298it [00:50, 6614.76it/s]


In [18]:
# sort games
for k, v in lookup.items():
    lookup[k] = sorted(v, key=lambda x: x[0])

In [19]:
# remove timespace
for k, v in lookup.items():
    lookup[k] = list(map(lambda x: x[1], v))

In [20]:
# insert end 
for k, v in lookup.items():
    lookup[k].append("*")
lookup

{443992607815436: [1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  '*'],
 6613300821617026: [0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  

In [21]:
class Step(object):
    def __init__(self, game_idx, outcome, path):
        self.game_idx = game_idx
        self.count = 0
        self.outcome = outcome
        self.left = None
        self.right = None
        self.end = None
        self.path = path

In [22]:
root = Step("ftd", outcome="start", path="")

data = lookup.values()

for d in data:
    curr = root

    for i, o in enumerate(d):
        if len(d) > 10:
            pass
        
        # scenario 1- win
        elif o == 1:
            if curr.right is None:  
                p = curr.path
                curr.right = Step(game_idx=i, outcome="win", path=p+'W')
            curr = curr.right
            curr.count += 1
        elif o == 0:
            if curr.left is None:
                p = curr.path
                curr.left = Step(i, outcome="loss", path=p+"L")
            curr = curr.left
            curr.count += 1
        else:
            if curr.end is None:
                p = curr.path
                curr.end = Step(i, outcome="end", path=p+'*')
            curr = curr.end
            curr.count += 1

In [23]:
journeys = {}

def f(x):

    if x.outcome == "end":
        print(x.game_idx, x.path, x.count)
        journeys[x.count] = x.path
    else:
        if x.left is not None:
            f(x.left)
        if x.right is not None:
            f(x.right)
        if x.end is not None:
            f(x.end)

In [24]:
f(root)

9 LLLLLLLLL* 11
8 LLLLLLLL* 9
9 LLLLLLLWL* 3
9 LLLLLLLWW* 1
8 LLLLLLLW* 1
7 LLLLLLL* 21
9 LLLLLLWLL* 3
8 LLLLLLWL* 1
9 LLLLLLWWW* 1
8 LLLLLLWW* 1
7 LLLLLLW* 4
6 LLLLLL* 30
8 LLLLLWLL* 3
9 LLLLLWLWL* 1
9 LLLLLWLWW* 1
7 LLLLLWL* 9
9 LLLLLWWLL* 1
9 LLLLLWWLW* 1
8 LLLLLWWL* 2
8 LLLLLWWW* 1
7 LLLLLWW* 1
6 LLLLLW* 5
5 LLLLL* 49
8 LLLLWLLL* 1
8 LLLLWLLW* 1
7 LLLLWLL* 4
9 LLLLWLWLW* 2
8 LLLLWLWL* 2
9 LLLLWLWWL* 1
8 LLLLWLWW* 2
7 LLLLWLW* 3
6 LLLLWL* 7
9 LLLLWWLLL* 1
9 LLLLWWLWL* 1
9 LLLLWWWLL* 2
9 LLLLWWWWW* 1
8 LLLLWWWW* 1
7 LLLLWWW* 1
6 LLLLWW* 1
5 LLLLW* 8
4 LLLL* 104
9 LLLWLLLLL* 1
8 LLLWLLLL* 5
9 LLLWLLLWL* 1
8 LLLWLLLW* 1
7 LLLWLLL* 7
8 LLLWLLWL* 3
6 LLLWLL* 12
9 LLLWLWLLL* 1
8 LLLWLWLL* 2
9 LLLWLWLWW* 1
7 LLLWLWL* 4
9 LLLWLWWWL* 1
9 LLLWLWWWW* 2
6 LLLWLW* 4
5 LLLWL* 14
8 LLLWWLLL* 2
8 LLLWWLLW* 1
7 LLLWWLL* 1
9 LLLWWLWLL* 1
8 LLLWWLWL* 1
9 LLLWWLWWW* 1
7 LLLWWLW* 2
6 LLLWWL* 3
9 LLLWWWLLL* 1
8 LLLWWWLL* 1
9 LLLWWWLWL* 1
7 LLLWWWL* 3
9 LLLWWWWWL* 1
8 LLLWWWWW* 1
7 LLLWWWW* 1
6 LLLWWW* 1


In [25]:
games_played = sum(journeys.keys())
games_played

2988

In [26]:
for k in sorted(journeys.keys()):
    print(journeys[k], round(k / games_played, 3))

WWWWWWWW* 0.0
WWWWWLWW* 0.001
WWWWWW* 0.001
WWWWWWW* 0.001
WWWLW* 0.002
WWLWL* 0.002
WWWWW* 0.002
WWWWL* 0.003
LWLLL* 0.003
WWWLL* 0.003
WWLW* 0.004
WWLLL* 0.004
LLLW* 0.004
WWWW* 0.005
WWWL* 0.005
WLW* 0.005
LLWLL* 0.006
WLLLL* 0.006
WWLL* 0.007
LLLLLLL* 0.007
WLWL* 0.008
WWW* 0.009
LLLLLL* 0.01
LLWL* 0.012
LWLL* 0.013
WLLL* 0.014
LLW* 0.015
LLLLL* 0.016
WW* 0.018
WWL* 0.02
LWL* 0.022
LW* 0.028
WLL* 0.029
LLLL* 0.035
WL* 0.049
W* 0.061
LLL* 0.065
LL* 0.169
L* 0.333
